<a href="https://colab.research.google.com/github/58191554/PointNet-Project/blob/TongZhen-branch/Copy_of_PointNetClass_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PointNet

This is an implementation of [PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation](https://arxiv.org/abs/1612.00593) using PyTorch.


## Getting started

Don't forget to turn on GPU if you want to start training directly. 


**Runtime** -> **Change runtime type**-> **Hardware accelerator**



In [1]:
import numpy as np
import math
import random
import os
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

In [2]:
!pip install path.py;
from path import Path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
random.seed = 42

In [4]:
!wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip

--2023-04-24 00:52:11--  http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Resolving 3dvision.princeton.edu (3dvision.princeton.edu)... 128.112.136.74
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip [following]
--2023-04-24 00:52:11--  https://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
Connecting to 3dvision.princeton.edu (3dvision.princeton.edu)|128.112.136.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473402300 (451M) [application/zip]
Saving to: ‘ModelNet10.zip.1’

ModelNet10.zip.1    100%[===================>] 451.47M  76.4MB/s    in 5.7s    

2023-04-24 00:52:17 (78.6 MB/s) - ‘ModelNet10.zip.1’ saved [473402300/473402300]



Download the [dataset](http://3dvision.princeton.edu/projects/2014/3DShapeNets/) directly to the Google Colab Runtime. It comprises 10 categories, 3,991 models for training and 908 for testing.

In [ ]:
!unzip -q ModelNet10.zip;

In [ ]:
path = Path("ModelNet10")

In [ ]:
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

This dataset consists of **.off** files that contain meshes represented by *vertices* and *triangular faces*. 

We will need a function to read this type of files:

In [ ]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [ ]:
with open(path/"bed/train/bed_0001.off", 'r') as f:
  verts, faces = read_off(f)

In [10]:
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [11]:
len(x)

2095

Don't be afraid of this huge function. It's  just to display animated rotation of meshes and point clouds.

In [12]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [13]:
visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()

This mesh definitely looks like a bed.

In [14]:
visualize_rotate([go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')]).show()

Unfortunately, that's not the case for its vertices. It would be difficult for PointNet to classify point clouds like this one.

First things first, let's write a function to accurately visualize point clouds so we could see vertices better.

In [15]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()
    

In [16]:
pcshow(x,y,z)

## Transforms

As we want it to look more like a real bed, let's write a function to sample points on the surface uniformly.

 ### Sample points

In [17]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    

In [18]:
pointcloud = PointSampler(3000)((verts, faces))

In [19]:
pcshow(*pointcloud.T)

This pointcloud looks much more like a bed!

### Normalize

Unit sphere

In [20]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [21]:
norm_pointcloud = Normalize()(pointcloud)

In [22]:
pcshow(*norm_pointcloud.T)

Notice that axis limits have changed.

### Augmentations

Let's add *random rotation* of the whole pointcloud and random noise to its points.

In [23]:
class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

In [24]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)

In [25]:
pcshow(*noisy_rot_pointcloud.T)

### ToTensor

In [26]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [27]:
ToTensor()(noisy_rot_pointcloud)

tensor([[ 0.4239,  0.2259,  0.0682],
        [ 0.1924,  0.2541,  0.0434],
        [ 0.6200,  0.2670,  0.2357],
        ...,
        [ 0.2716, -0.5446,  0.0986],
        [ 0.4325,  0.1668,  0.0154],
        [ 0.2137, -0.6406,  0.3784]], dtype=torch.float64)

In [28]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

## Dataset

Now we can create a [custom PyTorch Dataset](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [29]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

Transforms for training. 1024 points per cloud as in the paper!

In [30]:
train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])

In [31]:
train_ds = PointCloudData(path, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', transform=train_transforms)

In [32]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()};
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

In [33]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())
print('Class: ', inv_classes[train_ds[0]['category']])

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
Sample pointcloud shape:  torch.Size([1024, 3])
Class:  bathtub


In [34]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

## Model

#### T-Net
The T-Net module is a type of Spatial Transformer Network (STN) that learns a kxk transformation matrix for a given point cloud, which is then used to transform the point cloud to a canonical pose. It consists of two parts: a convolutional network and a fully connected network. The convolutional network maps the input point cloud to a feature space, consisting of a series of convolutional layers with batch normalization and ReLU activation. The fully connected network takes the feature space and learns the transformation matrix, consisting of fully connected layers with batch normalization and ReLU activation. Finally, the T-Net applies the transformation matrix to the input point cloud to transform it to a canonical pose.
![T-net](https://github.com/58191554/PointNet-Project/blob/main/img/T-net_pipeline.drawio.png?raw=true)

In [35]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class Tnet(nn.Module):
    """
    T-Net is a type of spatial transformer network (STN) that learns a kxk transformation matrix
    for a given point cloud. The matrix is then used to transform the point cloud to a canonical
    pose. It consists of two parts: a convolutional network and a fully connected network.
    The convolutional network maps the input point cloud to a feature space and the fully connected
    network learns the transformation matrix from the feature space.
    """
    def __init__(self, hidden_sizes_conv=[64, 128, 1024], hidden_sizes_fc=[512, 256], k=3):
        super().__init__()
        self.k=k
        self.hidden_sizes_conv=hidden_sizes_conv
        self.hidden_sizes_fc=hidden_sizes_fc
        
        self.conv = self._build_conv()
        self.fc = self._build_fc()
  
    def _build_conv(self):
        ########################################################################
        # TODO: Builds the convolutional network that maps the input point cloud 
        # to a feature space. The hidden dimension is hidden_sizes_conv
        #  
        # Hint: consisting of a series of convolutional layers with batch 
        # normalization and ReLU activation.
        #   The convolution layers is in following structure:
        #   [conv1d]-> [Batch Norm Layer] -> [ReLU]-> [conv1d]-> ...
        #   Take the integer in hidden_size_conv for convolution1d layer
        #   size and batch_norm layer size
        ########################################################################
        layers = []
        prev_size = self.k
        for layer_id, size in enumerate(self.hidden_sizes_conv):
            bn = nn.BatchNorm1d(size)
            conv = nn.Conv1d(prev_size, size,1)
            layers.append(conv)
            layers.append(bn)
            layers.append(nn.ReLU())
            prev_size = size
        ########################################################################
        return nn.Sequential(*layers)
  
    def _build_fc(self):
        ########################################################################
        # TODO:  the fully connected network that takes the feature space and 
        # learns the transformation matrix. 
        #   The hidden_layers according to hidden_sizes_fc
        # 
        # Hint: the fully connected structur is as follows:
        #   [Fully Connected Layer]-> [Batch Norm Layer] -> [ReLU]-> [Fully Connected Layer]-> ...
        #   Each layer take the integer in hidden_sizes_fc as dimention
        #   Add one addiontional Linear layer to reach matrix-shape as last layer
        ########################################################################
        layers = []
        prev_size = self.hidden_sizes_conv[-1]
        for layer_id, size in enumerate(self.hidden_sizes_fc):
            bn = nn.BatchNorm1d(size)
            fc = nn.Linear(prev_size, size)
            layers.append(fc)
            layers.append(bn)
            layers.append(nn.ReLU())
            prev_size = size
        layers.append(nn.Linear(self.hidden_sizes_fc[-1],self.k**2))
        ########################################################################
        return nn.Sequential(*layers)
      

    def forward(self, input):
        ########################################################################
        # TODO: Performs the forward pass of the T-Net. 
        # It first applies the convolutional network to the input point cloud 
        # to obtain a feature space. 
        # Then, it applies the fully connected network to the feature space to 
        # obtain the kxk transformation matrix. Finally, it applies the
        # transformation matrix to the input point cloud to transform it to a 
        # canonical pose.
        # input: [batch, k=3, N], N is the points number in a object
        # output: [batch, k, k]
        # 
        # Hint: the forward structure is as follows:
        # [ConvLayers]->[MaxPooling]->[Flatten]->[Fully Connected Layers]->[theta_Matrix + identity]
        #   The identity require gradient
        ########################################################################
        # input.shape (bs,n,3)
        bs = input.size(0)
        
        xb = self.conv(input)   
        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)
        xb = self.fc(flat)
      
        init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
        if xb.is_cuda:
          init=init.cuda()
        matrix = xb.view(-1,self.k,self.k) + init        
        return matrix

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

torch.manual_seed(42)
test_t_net = Tnet()

print("Getting T-Net parameters number:",end=(""))
if count_parameters(test_t_net)!=803081:
    print("Error")
    print("Your answer is: ", count_parameters(test_t_net))

assert count_parameters(test_t_net)==803081
print("passed")


Getting T-Net parameters number:passed


In [37]:
torch.manual_seed(42)
x1 = torch.randn(3, 3, 5)

y1 = torch.tensor([[[ 1.4712e+00,  1.1447e+00,  6.5780e-02],
         [ 2.6862e-01,  1.5355e+00, -7.9635e-01],
         [-3.1744e-01,  4.8485e-01,  1.2669e+00]],

        [[ 1.0652e+00, -2.9729e-02, -9.1289e-04],
         [-2.0753e-01,  1.6646e+00,  5.0989e-01],
         [-2.5312e-01,  7.1402e-01,  8.2575e-01]],

        [[ 1.3445e+00,  6.7090e-01, -4.4554e-01],
         [ 2.4452e-01,  1.1833e+00, -5.8614e-01],
         [-5.3094e-02, -1.3413e-01,  9.4217e-01]]])

pred_y1 = test_t_net(x1)

print("Getting T-Net out:", end = (""))
if not torch.allclose(y1, pred_y1, rtol=1e-03, atol=1e-03):
    print("Error")
    print("Your answer is:", pred_y1)
    print("The expected answer is:", y1)

assert torch.allclose(y1, pred_y1, rtol=1e-03, atol=1e-03),  "different y_pred and y"
print("passed")

Getting T-Net out:passed


### Transfrom Class
The Transform class is every thing before last MLPs in PointNet. It is a neural network architecture that uses two pairs of spatial transform net (STN) and shared MLP layers to extract global features from a point cloud data of (nx3) shape. The STN is implemented using the T-Net and computes the 3x3 transform matrix, which is then multiplied with the input point cloud to get a transformed point cloud of the same shape. The transformed point cloud is then input into the shared MLP layers along with the feature transform matrix. The output from the shared MLP layers is max pooled along the feature dimension to get a global feature vector. The output also includes the point and feature transform matrices.
![TransformNet](https://github.com/58191554/PointNet-Project/blob/main/img/PointNetStructureFromPaper.png?raw=true)

In [38]:
class Transform(nn.Module):
    def __init__(self, input_size=3, feature_size=64, sharedMLP1_layers=[64, 64], sharedMLP2_layers=[64, 128, 1024], batch_norm = True):
        """
        Transform class is all the pipeline to get a global feature
                 _____________________                                     _______________                 ___________________       _______________
                |                     |                                   |               |                |                 |     |               |                    
        x -->   |   input transform   | --> y (canonical point cloud) --> |  shared MLP   | --> feature -->|feature transform| --> |  shared MLP   | --> max pooling --> z
                |_____________________|                                   |_______________|                |_________________|     |  _____________|
        The transform class is a neural networknet architecture that go throught 2 pairs of spactial transform net and shared MLP.
        The STN is the T-Net that implement above, and the shared-MLP can be regarded as a one-dimensional convolutional layer.

        the input x as a point cloud data of (nx3) shape first compute the 3x3 transform matrix and multiplied with the transform matrix to get a (nx3) transformed point cloud.

        the last_activate bool is True when you want to add the last layer with activation function.
        """
        super().__init__()
        self.batch_norm = True
        
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)

        self.sharedMLP1 = self._build_sharedMLP(input_size, sharedMLP1_layers, last_activate=True)
        self.sharedMLP2 = self._build_sharedMLP(feature_size, sharedMLP2_layers, last_activate=False)

    def _build_sharedMLP(self, input_dim, sharedMLP_layers, last_activate = True):
        ########################################################################
        # TODO: Build the shared MLP layers. The sharedMLP_layers is a 
        # list of integer, each represents the output dimension of layer
        # Hint: 
        #   The structure is [Conv1d]->[Batch Norm]->[ReLU]->[Conv1d]->...
        #   The last sections of layer doesn't require ReLU
        ########################################################################
        layers = []
        prev_size = input_dim
        for layer_id, size in enumerate(sharedMLP_layers):
            layers.append(nn.Conv1d(prev_size, size, 1))

            if self.batch_norm:
                layers.append(nn.BatchNorm1d(size))

            if (layer_id < len(sharedMLP_layers)-1) or last_activate:
                layers.append(nn.ReLU())

            prev_size = size
        return nn.Sequential(*layers)
       
    def forward(self, input):     
    
        ########################################################################
        # TODO: Implement the code to multiply the transform matrix and the point
        # cloud. The transformed x should be the same shape of x 
        # 
        # input:[batch_size, 3, N] output:[batch_size, N], N is the number of points of a object
        # output: [batch_size, 1024]
        # Hint: 
        # 1. Get the transform matrix by the T-Net
        # 2. Batch matrix multiply the input x and transform matrix
        # 3. Input the data into the Shared MLP
        # 4. Batch matrix multiply the feature and the feature_transform matrix
        # 5. Input the output into the Shared MLP with feature dimension
        # 6. Maxpooling along the feature dimension
        # 7. output the output data, points transform matrix, and the feature
        #       transform matrix
        ########################################################################
        matrix3x3 = self.input_transform(input)     #[batch_size, 3, 3]
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)     #[batch_size, 3, 1024]
        xb = self.sharedMLP1(xb)

        matrix64x64 = self.feature_transform(xb)     #[batch_size, 64, 64]
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)     #[batch_size, 64, 1024]
        xb = self.sharedMLP2(xb)

        xb = nn.MaxPool1d(xb.size(-1))(xb)     #[batch_size, 1024, 1]
        output = nn.Flatten(1)(xb)     #[batch_size, 1024]
        ########################################################################
        return output, matrix3x3, matrix64x64

In [39]:
torch.manual_seed(42)
test_transfrom_net = Transform()

test_tranform_net_param_num = count_parameters(test_transfrom_net)
print("Getting transforom net parameters number:", end=(""))
if test_tranform_net_param_num!=2812105:
    print("Error")
    print("Your answer is : ", count_parameters(test_transfrom_net))
    print("Expected answer = ", 2812105)

assert count_parameters(test_transfrom_net)==2812105
print("passed")


torch.manual_seed(42)
x2 = torch.randn(2, 3, 5)
pred_y2, pred_mat1, pred_mat2 = test_transfrom_net(x2)

mat1 = torch.tensor([[[ 1.0655,  0.4169,  0.1452],
         [ 0.0240,  1.7885, -0.6011],
         [-0.5582,  0.6857,  1.0256]],

        [[ 1.5976,  0.8703, -0.4317],
         [ 0.2260,  1.2361,  0.0457],
         [ 0.0986,  0.0844,  1.0267]]])

print("Getting value of  mat1:", end = (""))
if not torch.allclose(mat1, pred_mat1, rtol=1e-03, atol=1e-03):
    print("Error")
    print("Your answer mat1 is \n", pred_mat1)
    print("The expected answer is \n", mat1)

assert torch.allclose(mat1, pred_mat1, rtol=1e-03, atol=1e-03),  "different pred_mat1 and mat1"
print("passed")

Getting transforom net parameters number:passed
Getting value of  mat1:passed


### PointNet Classifier
The code defines a PyTorch module called PointNet for classifying point cloud data. The PointNet module includes a Transform class, which takes in 3D point cloud data as input and generates global features and transformation matrices. The global features are then passed through a multi-layer perceptron (MLP) to generate scores for classification. The MLP consists of linear layers, batch normalization, ReLU activation, and dropout layers. The PointNet module outputs the logsoftmax of the scores along with the 3x3 and 64x64 transformation matrices generated by the Transform class. The PointNet module can be customized with different layer configurations, batch normalization, and dropout rates.

In [40]:
class PointNet(nn.Module):
    def __init__(self, sharedMLP1_layers=[64, 64], sharedMLP2_layers=[64, 128, 1024], classes = 10, batch_norm = True, dropout_rate = 0.3):
        """
        Point Net the whole neural network for the classification of point cloud data
                    _________                         ___     
        input x--->|Transform|---> global feature--->|MLP|---> scores
                   |_________|                       |___|
            args:   sharedMLP1_layers is the first shared MLP in Transform class
                    sharedMLP2_layers is the second shared MLP in Transorm class
                The MLP has the structure of 
                    [Linear] -> [Batch Norm] -> [ReLU] -> [Dropout] -> [Linear] -> ... -> [Linear] -> [Batch Norm] -> [ReLU] -> [Linear of class size]

                
        """
        super().__init__()
        self.transform = Transform(input_size=3, feature_size=64, sharedMLP1_layers=sharedMLP1_layers, sharedMLP2_layers=sharedMLP2_layers)
        self.batch_norm = batch_norm
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(dropout_rate)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def _build_fc(self, input_dim, fc_layers, cls_num, dropout_rate):
        layers = []
        prev_size = input_dim
        for layer_id, size in enumerate(fc_layers):
            layers.append(nn.Linear(prev_size, size, 1))

            if self.batch_norm:
                layers.append(nn.BatchNorm1d(size))

            if layer_id < len(fc_layers):
                layers.append(nn.ReLU())

            if layer_id < len(fc_layers)-1:
                layers.append(nn.Dropout(dropout_rate))
            prev_size = size
        
        layers.append(nn.Linear(cls_num))
        return nn.Sequential(*layers)
        

    def forward(self, input):
        ########################################################################
        # TODO: get the output y, 3x3 transform matrix and 64x64 transform
        # matrix from self.transform net.
        # Then, y->[fc1]->[bn1]->[relu]->[fc2]->[dropout]->[bn2]->[relu]->[fc3]->z
        # return logsoftmax(z), 3x3 transform matrix and 64x64 transform matrix
        # input: [batch, 3, N]
        # output: []
        ########################################################################

        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [41]:
torch.manual_seed(42)
test_point_net = PointNet()

test_point_net_param_num = count_parameters(test_point_net)
if test_point_net_param_num!=3472339:
    print("Error")
    print("test_transfrom_net parameters number = ", count_parameters(test_point_net))
    print("Difference = ", torch.absolute(torch.tensor(test_point_net_param_num-3472339)))

assert count_parameters(test_point_net)==3472339


torch.manual_seed(42)
x3 = torch.randn(3, 3, 5)
w, pred_mat3x3, pred_matfxf = test_point_net(x3)
print(w.shape)

mat3x3 = torch.tensor([[[ 1.4712e+00,  1.1447e+00,  6.5780e-02],
         [ 2.6862e-01,  1.5355e+00, -7.9635e-01],
         [-3.1744e-01,  4.8485e-01,  1.2669e+00]],

        [[ 1.0652e+00, -2.9729e-02, -9.1289e-04],
         [-2.0753e-01,  1.6646e+00,  5.0989e-01],
         [-2.5312e-01,  7.1402e-01,  8.2575e-01]],

        [[ 1.3445e+00,  6.7090e-01, -4.4554e-01],
         [ 2.4452e-01,  1.1833e+00, -5.8614e-01],
         [-5.3094e-02, -1.3413e-01,  9.4217e-01]]])

print("Getting value of mat3x3:", end = (""))
if not torch.allclose(mat3x3, pred_mat3x3, rtol=1e-03, atol=1e-03):
    print("Error")
    print("Your answer mat3x3 is : \n", mat1)
    print("Expected value is : \n", pred_mat1)
    print("Difference = ", torch.norm(pred_mat3x3- mat3x3))
assert torch.allclose(pred_mat3x3, mat3x3, rtol=1e-03, atol=1e-03),  "different pred_mat1 and mat1"
print("passed")

torch.Size([3, 10])
Getting value of mat3x3:passed


### Orthogonal Regulation Loss
We constrain the feature transformation matrix to be
close to orthogonal matrix. A matrix multiply itself close to identity when the columns are orthogonal.An orthogonal transformation will not lose
information in the input, thus is desired. We find that by
adding the regularization term, the optimization becomes
more stable and our model achieves better performance.   
$$L_{reg}=||I-AA^T||_F^2$$
In the following coding part, consider:
$$L = criterion(\cdot)+\alpha\frac{\sum_{j = 1}^{Batch}||I-A_jA_j^T||_F^2}{Batch}$$

In [42]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    """
    Add a loss on the transform theta matrix 
    """
    criterion = torch.nn.CrossEntropyLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)       #identity matrix repeat batch_size times
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    #############################################################################
    # TODO: Get the loss of mat3x3 and mat64x64 and add to the loss
    # input: outputs is a tensor of [batch, class_number]
    #   labels is also a tensor of [batch, class_number]
    #   m3x3 is the learned transform matrix [batch, 3, 3]
    #   m64x63 is the learned transform matrix [batch, 64, 64] 
    #   alpha is the coeffient for loss
    # Hint: use torch.bmm() to calculate batch matrix multilplication
    ##############################################################################
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))       #Loss:I-A.t@A
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    loss = criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)
    ##############################################################################
    return loss

In [43]:
torch.manual_seed(42)
x4 = torch.log(torch.randn((3, 10)))
torch.manual_seed(42)
labels_ = torch.randn((3, 10))
torch.manual_seed(42)
M1 = torch.randn((3, 3, 3))
torch.manual_seed(42)
M2 = torch.randn(3, 64, 64)
x4_onehot = torch.nn.functional.one_hot(x4.argmax(dim=1), num_classes=10).float()
labels_onehot = torch.nn.functional.one_hot(labels_.argmax(dim=1), num_classes=10).float()

pred_loss = pointnetloss(x4_onehot, labels_onehot, M1, M2)

loss_answer = torch.tensor(2.503323554992676)

print("Getting the loss:", end=(""))
if not torch.allclose(pred_loss, loss_answer, rtol=1e-03, atol=1e-03):
    print("Error")
    print("Your answer is:", pred_loss)
    print("Expected answer = ", loss_answer)

assert torch.allclose(pred_loss, loss_answer, rtol=1e-03, atol=1e-03)
print("passed")

Getting the loss:passed


## Training loop

You can find a pretrained model [here](https://drive.google.com/open?id=1nDG0maaqoTkRkVsOLtUAR9X3kn__LMSL)

In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [45]:
pointnet = PointNet()
pointnet.to(device);

In [46]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

In [47]:
from tqdm.notebook import trange, tqdm

In [48]:
class Experiment:
    def __init__(self, train_data, val_data, model: nn.Module,
                 lr: float, save=True):
        self.train_data = train_data
        self.val_data = val_data
        self.model = model.cuda()
        self.optimizer = torch.optim.Adam(pointnet.parameters(), lr=lr)
        self.save=save
        self.loss=[]

    def train(self, epochs):
        epoch_iterator = trange(epochs)
        for epoch in epoch_iterator: 
            self.model.train()
            running_loss = 0.0
            data_iterator = tqdm(self.train_data)
            for i, data in enumerate(data_iterator, 0):
                inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                self.optimizer.zero_grad()
                outputs, m3x3, m64x64 = self.model(inputs.transpose(1,2))

                loss = self.getloss(outputs, labels, m3x3, m64x64)
                loss.backward()
                self.loss.append(loss.item())
                self.optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 10 == 9:    # print every 10 mini-batches
                    data_iterator.set_postfix(loss=running_loss / 10)
                    running_loss = 0.0

            self.model.eval()
            correct = total = 0

            # validation
            if self.val_data:
                with torch.no_grad():
                    for data in self.val_data:
                        inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                        outputs, __, __ = self.model(inputs.transpose(1,2))
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                val_acc = 100. * correct / total
                #print('Valid accuracy: %d %%' % val_acc)
                epoch_iterator.set_postfix(val_acc=val_acc)
            if self.save:
                torch.save(self.model.state_dict(), "save_"+str(epoch)+".pth")

    def getloss(self, outputs, labels, m3x3, m64x64, alpha = 0.0001):
        criterion = torch.nn.NLLLoss()
        bs=outputs.size(0)
        id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)       #identity matrix repeat batch_size times
        id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
        if outputs.is_cuda:
            id3x3=id3x3.cuda()
            id64x64=id64x64.cuda()
        diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))       #Loss:I-A.t@A
        diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
        return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)
    
    def plt_loss(self):
      x1 = range(0, len(self.loss))
      y1 = self.loss
      plt.plot(x1, y1, 'o-')
      plt.title('Test loss vs. epoches')
      plt.ylabel('Test loss')
      plt.show()

In [49]:
exp = Experiment(train_loader, valid_loader, pointnet, 0.001, save=False)
exp.train(1)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
exp.plt_loss()

## Test

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
pointnet = PointNet()
pointnet.load_state_dict(torch.load('save.pth'))
pointnet.eval();

In [ ]:
all_preds = []
all_labels = []
with torch.no_grad():
    for i, data in enumerate(valid_loader):
        print('Batch [%4d / %4d]' % (i+1, len(valid_loader)))
                   
        inputs, labels = data['pointcloud'].float(), data['category']
        outputs, __, __ = pointnet(inputs.transpose(1,2))
        _, preds = torch.max(outputs.data, 1)
        all_preds += list(preds.numpy())
        all_labels += list(labels.numpy())
        


In [ ]:
cm = confusion_matrix(all_labels, all_preds);
cm

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

# function from https://deeplizard.com/learn/video/0LhiS6yu2qQ
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, list(classes.keys()), normalize=True)

In [ ]:
plt.figure(figsize=(8,8))
plot_confusion_matrix(cm, list(classes.keys()), normalize=False)